In [1]:
!pip3 install pandas
!pip3 install seaborn
!pip3 install --upgrade tensorflow-gpu
!pip3 install bayesian-optimization

from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import glob
import os
import math
import pickle
import time
import sys

import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tensorflow.keras import models, layers
from tensorflow.keras.layers import Lambda, Input, Dense, Dropout, Conv1D, MaxPooling1D, Flatten, GlobalAveragePooling1D, BatchNormalization
from tensorflow.keras.losses import mse
from tensorflow.keras.optimizers.legacy import Nadam
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import initializers
from tensorflow.keras import regularizers

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, mean_squared_error, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

from bayes_opt import BayesianOptimization

%matplotlib inline

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable


2022-11-24 19:28:05.548335: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-24 19:28:05.739690: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-24 19:28:05.739709: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-24 19:28:06.658206: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

In [2]:
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction = 0.333)
sess = tf.compat.v1.Session(config = tf.compat.v1.ConfigProto(gpu_options = gpu_options))

2022-11-24 19:28:07.589155: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-24 19:28:07.619515: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-24 19:28:07.619616: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-11-24 19:28:07.619684: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: li

# BoT IoT Data Pre-Processing

In [3]:
# Traffic information

path = '../botiot/'
all_files = glob.glob(os.path.join(path , '*.csv'))

files_list = []
for file in all_files:
    df = pd.read_csv(file, index_col = None, encoding = 'utf-8', sep = ',', low_memory = False)
    files_list.append(df)

df_traffic = pd.concat(files_list, axis = 0, ignore_index = True)
df_traffic = df_traffic.drop(columns=['pkSeqID', 'stime', 'flgs', 'flgs_number', 'saddr', 'sport', 'daddr', 'dport', 'subcategory'])
df_traffic['proto'] = df_traffic['proto'].map({'tcp': 1, 'arp': 2, 'udp': 3, 'icmp': 4, 'ipv6-icmp': 5})
df_traffic['state'] = df_traffic['state'].map({'REQ': 1, 'RST': 2, 'ACC': 3, 'CON': 4, 'INT': 5, 'URP': 6, 'FIN': 7, 'NRS': 8, 'ECO': 9, 'TST': 10, 'MAS': 11})

In [4]:
# Attacks

df_attack = df_traffic.query('attack == 1')
df_attack = df_attack.drop(columns=['attack'])
df_attack['category'] = df_attack['category'].map({'DDoS': 0, 'DoS': 1, 'Reconnaissance': 2, 'Theft': 3})

df_ddos = df_attack.query('category == 0')
label_ddos = df_ddos.pop('category')

df_dos = df_attack.query('category == 1')
label_dos = df_dos.pop('category')

df_rec = df_attack.query('category == 2')
label_rec = df_rec.pop('category')

df_theft = df_attack.query('category == 3')
label_theft = df_theft.pop('category')

print(len(df_ddos))
print(len(df_dos))
print(len(df_rec))
print(len(df_theft))

1926624
1650260
91082
79


In [5]:
scaler = MinMaxScaler()

df_ddos_norm = scaler.fit_transform(df_ddos)
df_dos_norm = scaler.fit_transform(df_dos)
df_rec_norm = scaler.fit_transform(df_rec)
df_theft_norm = scaler.fit_transform(df_theft)

# CNN - Attack Classification

In [6]:
batch_size = 50
number_features = 23
learning_rate = 0.008
epochs = 25

dict_params = { 'learning_rate': learning_rate, 'batch_size': int(32 * batch_size), 'epochs': int(epochs) }
pbounds = { 'learning_rate': (0.000001, 0.1), 'batch_size': (1, 4.001), 'epochs': (1, 50) }

In [7]:
# Train set

len_ddos_train = int(0.7 * len(df_ddos_norm))
X_ddos_train = df_ddos_norm[:len_ddos_train]

len_dos_train = int(0.7 * len(df_dos_norm))
X_dos_train = df_dos_norm[:len_dos_train]

len_rec_train = int(0.7 * len(df_rec_norm))
X_rec_train = df_rec_norm[:len_rec_train]

len_theft_train = int(0.7 * len(df_theft_norm))
X_theft_train = df_theft_norm[:len_theft_train]

np_train = np.concatenate([X_ddos_train, X_dos_train, X_rec_train, X_theft_train])
np_label_train = np.concatenate([label_ddos[:len_ddos_train], label_dos[:len_dos_train], label_rec[:len_rec_train], 
                                 label_theft[:len_theft_train]])

df_train = pd.DataFrame(np_train)
df_label_train = pd.DataFrame(np_label_train)

X_train = df_train.to_numpy()
Y_train = df_label_train.to_numpy()

print(len(X_train))

2567630


/tmp/ipykernel_31412/885869611.py:16: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  np_label_train = np.concatenate([label_ddos[:len_ddos_train], label_dos[:len_dos_train], label_rec[:len_rec_train],
/tmp/ipykernel_31412/885869611.py:17: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  label_theft[:len_theft_train]])


In [8]:
# Test set

len_ddos_test = len_ddos_train + int(0.15 * len(df_ddos_norm))
X_ddos_test = df_ddos_norm[len_ddos_train : len_ddos_test]

len_dos_test = len_ddos_train + int(0.15 * len(df_dos_norm))
X_dos_test = df_dos_norm[len_dos_train : len_dos_test]

len_rec_test = len_rec_train + int(0.15 * len(df_rec_norm))
X_rec_test = df_rec_norm[len_rec_train : len_rec_test]

len_theft_test = len_theft_train + int(0.15 * len(df_theft_norm))
X_theft_test = df_theft_norm[len_theft_train : len_theft_test]

np_test = np.concatenate([X_ddos_test, X_dos_test, X_rec_test, X_theft_test])
np_label_test = np.concatenate([label_ddos[len_ddos_train : len_ddos_test], label_dos[len_dos_train : len_dos_test], 
                                 label_rec[len_rec_train : len_rec_test], label_theft[len_theft_train : len_theft_test]])

df_test = pd.DataFrame(np_test)
df_label_test = pd.DataFrame(np_label_test)

X_test = df_test.to_numpy()
Y_test = df_label_test.to_numpy()

print(len(X_test))

743659


/tmp/ipykernel_31412/3168015556.py:16: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  np_label_test = np.concatenate([label_ddos[len_ddos_train : len_ddos_test], label_dos[len_dos_train : len_dos_test],
/tmp/ipykernel_31412/3168015556.py:17: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  label_rec[len_rec_train : len_rec_test], label_theft[len_theft_train : len_theft_test]])


In [9]:
# Validation set

X_ddos_val = df_ddos_norm[len_ddos_test:]
X_dos_val = df_dos_norm[len_dos_test:]
X_rec_val = df_rec_norm[len_rec_test:]
X_theft_val = df_theft_norm[len_theft_test:]

np_val = np.concatenate([X_ddos_val, X_dos_val, X_rec_val, X_theft_val])
np_label_val = np.concatenate([label_ddos[len_ddos_test:], label_dos[len_dos_test:], label_rec[len_rec_test:], 
                               label_theft[len_theft_test:]])

df_val = pd.DataFrame(np_val)
df_label_val = pd.DataFrame(np_label_val)

X_val = df_val.to_numpy()
Y_val = df_label_val.to_numpy()

print(len(X_val))

356756


/tmp/ipykernel_31412/3487149240.py:9: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  np_label_val = np.concatenate([label_ddos[len_ddos_test:], label_dos[len_dos_test:], label_rec[len_rec_test:],
/tmp/ipykernel_31412/3487149240.py:10: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  label_theft[len_theft_test:]])


In [10]:
X_train_CNN = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test_CNN = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
X_val_CNN = np.reshape(X_val, (X_val.shape[0], X_val.shape[1], 1))

Y_train_CNN = Y_train
Y_test_CNN = Y_test
Y_val_CNN = Y_val

samples, feature, depth = X_train_CNN.shape

In [11]:
# CNN model with 1D convolutional layer

def CNN():
    model = models.Sequential()
    model.add(layers.Conv1D(32, 3, activation='relu', padding = 'same', kernel_initializer = 'he_uniform', input_shape=(feature, depth)))
    model.add(layers.MaxPooling1D(pool_size = 2, strides = 2))
    model.add(layers.Conv1D(64, 3, activation='relu', padding = 'same', kernel_initializer = 'he_uniform'))
    model.add(layers.MaxPooling1D(pool_size = 2, strides = 2))
    model.add(layers.Conv1D(64, 3, activation='relu', padding = 'same', kernel_initializer = 'he_uniform'))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(4, activation='softmax'))
    
    return model

In [12]:
reduce_lr = ReduceLROnPlateau(moniter = 'val_loss', factor = 0.1, patience = 10)

## Train

In [13]:
# Training step with Bayesian optimization

def training(X_train = X_train_CNN,
             Y_train = Y_train_CNN, 
             X_val = X_val_CNN, 
             Y_val = Y_val_CNN, 
             X_test = X_test_CNN, 
             Y_test = Y_test_CNN, 
             learning_rate = learning_rate, 
             epochs = epochs, 
             batch_size = batch_size,
             reduce_lr = reduce_lr):
    
    nadam = optimizers.Nadam(learning_rate = dict_params['learning_rate'], beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08, schedule_decay = 0.004)
    
    model = CNN()
    model.compile(loss = "sparse_categorical_crossentropy", optimizer = "nadam", metrics = ["accuracy"])
    
    history = model.fit(X_train, Y_train, 
                        epochs = dict_params['epochs'], 
                        batch_size = dict_params['batch_size'], 
                        validation_data = (X_val, Y_val),
                        callbacks = [reduce_lr])
    
    scores = model.evaluate(X_test, Y_test)
    
    print('loss:', scores[0])
    print('accuracy:', scores[1])
    return scores[1]

In [14]:
"""
# Bayesian optimization to choose the best hyperparameters

optimizer = BayesianOptimization(
    f = training,
    pbounds = pbounds,
    verbose = 2, 
    random_state = 1,
)

train_start = time.time()

optimizer.maximize(init_points = 5, n_iter = 5)

train_end = time.time()
train_time = train_end - train_start
print("Training time:", train_time)

print(optimizer.max)
"""

'\n# Bayesian optimization to choose the best hyperparameters\n\noptimizer = BayesianOptimization(\n    f = training,\n    pbounds = pbounds,\n    verbose = 2, \n    random_state = 1,\n)\n\ntrain_start = time.time()\n\noptimizer.maximize(init_points = 5, n_iter = 5)\n\ntrain_end = time.time()\ntrain_time = train_end - train_start\nprint("Training time:", train_time)\n\nprint(optimizer.max)\n'

In [ ]:
# Training step with the best hyperparameters

nadam = Nadam(learning_rate = 0.01946960847019594, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08, schedule_decay = 0.004)

model = CNN()
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "nadam", metrics = ["accuracy"])

train_start = time.time()

history = model.fit(X_train_CNN, Y_train_CNN, 
                    epochs = int(36.701988678683904), 
                    batch_size = int(32 * 3.179494411695542), 
                    validation_data = (X_val_CNN, Y_val_CNN),
                    callbacks = [reduce_lr])

train_end = time.time()
train_time = train_end - train_start
print("Training time:", train_time)

Train on 2567630 samples, validate on 356756 samples
Epoch 1/36


2022-11-24 19:28:43.371000: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-11-24 19:28:43.396309: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2022-11-24 19:28:43.461456: W tensorflow/c/c_api.cc:291] Operation '{name:'training/Nadam/dense_1/bias/v/Assign' id:439 op device:{requested: '', assigned: ''} def:{{{node training/Nadam/dense_1/bias/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](training/Nadam/dense_1/bias/v, training/Nadam/dense_1/bias/v/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will 

2567016/2567630 [============================>.] - ETA: 0s - loss: 0.0018 - accuracy: 0.9996

/home/andressa.amaral/.local/lib/python3.8/site-packages/keras/engine/training_v1.py:2333: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates
2022-11-24 19:30:53.746001: W tensorflow/c/c_api.cc:291] Operation '{name:'loss/mul' id:205 op device:{requested: '', assigned: ''} def:{{{node loss/mul}} = Mul[T=DT_FLOAT, _has_manual_control_dependencies=true](loss/mul/x, loss/dense_1_loss/value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


2567630/2567630 [==============================] - 137s 53us/sample - loss: 0.0018 - accuracy: 0.9996 - val_loss: 3.1716e-04 - val_accuracy: 0.9999 - lr: 0.0010
Epoch 2/36
2567630/2567630 [==============================] - 136s 53us/sample - loss: 1.7231e-04 - accuracy: 1.0000 - val_loss: 1.3340e-04 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 3/36
2567630/2567630 [==============================] - 135s 53us/sample - loss: 1.2122e-04 - accuracy: 1.0000 - val_loss: 2.9992e-04 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 4/36
2567630/2567630 [==============================] - 135s 53us/sample - loss: 1.1293e-04 - accuracy: 1.0000 - val_loss: 4.0085e-04 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 5/36
2567630/2567630 [==============================] - 136s 53us/sample - loss: 1.2794e-04 - accuracy: 1.0000 - val_loss: 9.4944e-04 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 6/36
2567630/2567630 [==============================] - 135s 53us/sample - loss: 1.1696e-04 - accuracy: 1.0000 - val_loss: 3.3969

## Test

In [ ]:
# Testing step

test_start = time.time()

Y_pred = model.predict(X_test_CNN)

test_end = time.time()
test_time = test_end - test_start
print("Testing time:", test_time)

In [ ]:
Y_pred_CNN = np.argmax(Y_pred, axis = 1)
Y_true_CNN = Y_test_CNN.astype(int)

## Metrics

In [ ]:
labels = ['DDoS', 'DoS', 'Reconaissance', 'Theft']

In [ ]:
# Multi classification metrics

acc = accuracy_score(Y_true_CNN, Y_pred_CNN) 
f1 = f1_score(Y_true_CNN, Y_pred_CNN, average = 'weighted')
pre = precision_score(Y_true_CNN, Y_pred_CNN, labels = None, pos_label = 1, average = 'weighted')
recall = recall_score(Y_true_CNN, Y_pred_CNN, labels = None, pos_label = 1, average = 'weighted', sample_weight = None)

In [ ]:
sys.stdout = open("../Results/bot_iot_cnn.txt", "a")

print(" ==== Test " + str(number_features) + " features ====")
print("Training time:" + str(train_time))
print("Testing time:" + str(test_time))
print("Accuracy:" + str(acc))
print("F1-score:" + str(f1))
print("Precision:" + str(pre))
print("Recall:" + str(recall))
#print("Hyper-parameters:" + str(optimizer.max))
print(classification_report(Y_true_CNN,Y_pred_CNN, target_names = labels))

# Naive Bayes

In [ ]:
# Model
nb = GaussianNB()

# Train
train_nb_start = time.time()

nb.fit(X_train, Y_train)

train_nb_end = time.time()
train_nb_time = train_nb_end - train_nb_start

# Test
test_nb_start = time.time()

Y_pred_nb = nb.predict(X_test)

test_nb_end = time.time()
test_nb_time = test_nb_end - test_nb_start

# Metrics
acc_nb = accuracy_score(Y_test, Y_pred_nb) 
f1_nb = f1_score(Y_test, Y_pred_nb, average = 'weighted')
pre_nb = precision_score(Y_test, Y_pred_nb, labels = None, pos_label = 1, average = 'weighted')
recall_nb = recall_score(Y_test, Y_pred_nb, labels = None, pos_label = 1, average = 'weighted', sample_weight = None)

print()
print(" ==== Naive Bayes " + str(number_features) + " features ====")
print("Training time:" + str(train_nb_time))
print("Testing time:" + str(test_nb_time))
print("Accuracy:" + str(acc_nb))
print("F1-score:" + str(f1_nb))
print("Precision:" + str(pre_nb))
print("Recall:" + str(recall_nb))
print(classification_report(Y_test,Y_pred_nb, target_names = labels))

# KNN

In [ ]:
# Model
knn = KNeighborsClassifier(n_neighbors = 50)

# Train
train_knn_start = time.time()

knn.fit(X_train, Y_train)

train_knn_end = time.time()
train_knn_time = train_knn_end - train_knn_start

# Test
test_knn_start = time.time()

Y_pred_knn = knn.predict(X_test)

test_knn_end = time.time()
test_knn_time = test_knn_end - test_knn_start

# Metrics
acc_knn = accuracy_score(Y_test, Y_pred_knn) 
f1_knn = f1_score(Y_test, Y_pred_knn, average = 'weighted')
pre_knn = precision_score(Y_test, Y_pred_knn, labels = None, pos_label = 1, average = 'weighted')
recall_knn = recall_score(Y_test, Y_pred_knn, labels = None, pos_label = 1, average = 'weighted', sample_weight = None)

print()
print(" ==== KNN " + str(number_features) + " features ====")
print("Training time:" + str(train_knn_time))
print("Testing time:" + str(test_knn_time))
print("Accuracy:" + str(acc_knn))
print("F1-score:" + str(f1_knn))
print("Precision:" + str(pre_knn))
print("Recall:" + str(recall_knn))
print(classification_report(Y_test,Y_pred_knn, target_names = labels))